In [1]:
from jupyter_dash import JupyterDash

In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [29]:
import pandas as pd
import numpy as np
import math
from modsim import *
import timeit

In [11]:
#############################################################################################
### Inputs we will need to get from the front-end:
sqft = 2400
roof_sqft = 850
Year_Blt = 1973

#roof_azimuth angle needs to be from due South
roof_azimuth = 40

#Daily_bool below directs whether to use stock Albany data or pull from NSRDB API (True = stock)
daily_bool = True

#The price per kWh ($0.1827 is average for Albany,NY)
price = 0.1827

#coordinates of Albany, NY (29.42412, -98.49363)
la=29.42412
lo=-98.49363

In [12]:
#############################################################################################
### If necessary (daily_bool = False), pull data from NSRDB
##  API key and info for M. Kollontai
#   Currently pulls data from 2019 and provides average and stDev for each month
def collect_ghi(la, lo):
    api_key = 'BUnBQIpFlpJZcCcqO2VeYuUMXjX7zCSGiVBNIIdH'
    attributes = 'ghi'
    year = '2019'
    lat, lon = la, lo
    leap_year = 'false'
    interval = '60'
    utc = 'false'
    name = 'Misha+Kollontai'
    reason= 'school_project'
    affiliation = 'CUNY+SPS'
    email = 'mkollontai@gmail.com'
    mailing_list = 'false'
    
    #combine all of the relevant information into the API-specified URL
    url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=name, email=email, mailing_list=mailing_list, affiliation=affiliation, reason=reason, api=api_key, attr=attributes)
    
    GHI_raw = pd.read_csv(url,skiprows = 2)
    #Set the index to the proper timestamps
    GHI_raw = GHI_raw.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
    temp = GHI_raw[['Month','Day','GHI']]
    daily = temp.groupby(['Month','Day']).sum()
    monthly_mean = daily.groupby(['Month']).mean()
    monthly_sd = daily.groupby(['Month']).std()
    monthly_ghi = pd.DataFrame(monthly_mean)
    monthly_ghi['STD'] = monthly_sd['GHI']
    return monthly_ghi.to_json()

In [23]:
#############################################################################################
### Depending on the 'daily_bool' variable, use stock Albany solar data or pull from NSRDB
if daily_bool:
    ## Data from the NSRDB. Daily data for Albany coordinates going back to 1998
    #  To be used as default data instead of querying the API
    Albany_GHI = pd.read_csv('https://raw.githubusercontent.com/zachalexander/capstone_cunysps/main/Project_Data_Files/Albany_GHI_Data.csv')
    Albany_GHI = Albany_GHI[['Month','Day','Mean','StDev']]
    Albany_GHI = Albany_GHI.set_index(['Month','Day'])
else:
    ## Use the function above to pull data on the coordinates in question
    monthly_ghi = pd.read_json(collect_ghi(la,lo))

In [15]:
### Data we pull from GitHub
##  Albany_Monthly_Use contains data from https://compareelectricity.com/locations/NY/Albany/12244
##  HHM/YearBuilt/SQFT_Ratios scale above data using data from https://www.eia.gov/consumption/residential/data/2015/c&e/pdf/ce1.2.pdf
Albany_Monthly_Use = pd.read_csv('https://raw.githubusercontent.com/zachalexander/capstone_cunysps/main/Data/Albany%20Monthly%20Average%20Use.csv')
Albany_Monthly_Use = Albany_Monthly_Use.set_index(['Month'])
#scale to household member number
HHM_Ratios = pd.read_csv('https://raw.githubusercontent.com/zachalexander/capstone_cunysps/main/Data/Albany%20Monthly%20Average%20Use%20-%20HHM.csv')
#scale to year house was built
YearBuilt_Ratios = pd.read_csv('https://raw.githubusercontent.com/zachalexander/capstone_cunysps/main/Data/Albany%20Monthly%20Average%20Use%20-%20YearBuilt.csv')
#scale to house square footage
SQFT_Ratios = pd.read_csv('https://raw.githubusercontent.com/zachalexander/capstone_cunysps/main/Data/Albany%20Monthly%20Average%20Use%20-%20Sqft.csv')

HHM_Ratios = HHM_Ratios.set_index(['Month'])
YearBuilt_Ratios = YearBuilt_Ratios.set_index(['Month'])
SQFT_Ratios = SQFT_Ratios.set_index(['Month'])
SQFT_Ratios

,Usage,1000,1499,1999,2499,2999,10000
Month,,,,,,,
1,905.2,425.444,706.056,995.72,1004.772,1140.552,1375.904
2,855.3,401.991,667.134,940.83,949.383,1077.678,1300.056
3,1095.2,514.744,854.256,1204.72,1215.672,1379.952,1664.704
4,875.2,411.344,682.656,962.72,971.472,1102.752,1330.304
5,723.6,340.092,564.408,795.96,803.196,911.736,1099.872
6,927.7,436.019,723.606,1020.47,1029.747,1168.902,1410.104
7,1245.4,585.338,971.412,1369.94,1382.394,1569.204,1893.008
8,1039.8,488.706,811.044,1143.78,1154.178,1310.148,1580.496
9,746.6,350.902,582.348,821.26,828.726,940.716,1134.832


In [88]:
#############################################################################################
#####   Define a system describing our solar panels and location ############################
#############################################################################################

def define_system(A=80,PR=0.8,lat=29.42412,long=-98.49363, azim = 1):
    '''Create a system object defining our solar panel system
    '''
    start = State(P = 0, C = 0)
    t0 = 0
    '''15 years worth of operation'''
    t_end = sim_years*12

    return System(start=start, t0=t0, t_end=t_end, A=A, PR=PR, lat=lat, long=long, azim = azim)
#############################################################################################

In [40]:
#############################################################################################
    ####   We must calculate the amount of power generated on on a given day by the panels. 
    ####   This number is influenced by the surface area of the panels, their efficiency, 
    ####   performance ratio and amount of exposure to sun they receive on that day. In our 
    ####   estimation of GHI on a given day, we will assume a normal distribution given the 
    ####   mean and stDev from the table we pulled from the NSRDB. The formula used below to 
    ####   calculate the actual yield is taken from 
    ####   (https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems) 
    ####   with the 'Annual average' value replaced with the GHI per day value calculated from the NSRDB data. 
#############################################################################################
####   Function to determine the daily yield of the panels   ################################
#############################################################################################
    ###      system     - pre-defined system defining the panels
    ###      month      - the month (1-12) for which the GHI is to be estimated
    ###      day        - the day of the month
    ###      daily_bool - whether or not we are using the stock Albany data or 2019 localized (T = stock data)
    
def days_yield(system,month,day,daily_bool):
    if daily_bool:
        ghi_day = np.random.normal(Albany_GHI.at[(month,day),'Mean'],Albany_GHI.at[(month,day),'StDev'])
    else:
        ghi_day = np.random.normal(monthly_ghi.iloc[month-1]['GHI'],monthly_ghi.iloc[month-1]['STD'])
    ghi_day = float(ghi_day)
    if ghi_day < 0:
        ghi_day = 0
    return (system.A*ghi_day*system.PR*system.azim)/1000
    
#############################################################################################

In [39]:
#############################################################################################
####   Function generating a value for the demand on our system in a month. 
#############################################################################################

            #*!*!*!*!*!*!* Need to account for how we determine demand*!*!*!*!*!*
    
    #  Currently taking average of 2 calulations using data from
    #  https://www.eia.gov/consumption/residential/data/2015/c&e/pdf/ce1.2.pdf
    #  to scale the info from 
    #  https://compareelectricity.com/locations/NY/Albany/12244
    #  on the demand in Albany per month
    #  Sqft calc scales the amounts to the appropriate sq ft bin
    #  Year Built calc does the same based on when the house was built
    #  If sqft or year built data not available, uses default numbers.

def month_demand_norm(month,sqft=0,year=0):
    # Sqft Lookup
    # should be set to zero if no info is available
    if sqft == 0:
        sqft_col = 0
    else:
        sqft_col = math.floor((sqft-1000)/500+2)
    sqft_month = SQFT_Ratios.iloc[month-1,sqft_col]
    
    # Year_Built Lookup
    # should be set to zero if no info is available
    if year == 0:
        year_col = 0
    else:
        year_col = math.floor((year-1950)/10+2)
    built_month = YearBuilt_Ratios.iloc[month-1,year_col]
    
    #std_d = tot_monthly * 0.15
    demand_month = (sqft_month+built_month)/2
    #if demand_month < 0:
    #    demand_month = 0
    return demand_month
    
#############################################################################################

In [57]:
#############################################################################################
####    Function calculating the balance at the end of a month ##############################
#############################################################################################

def calc_month(system, month):
    #2% yearly increase in electricity rates
    yearly_increase = 1.02
    year = math.floor(month/12)

    month_mod = (month % 12)
    if month_mod == 0:
        month_mod = 12
        
    if month_mod in [1,3,5,7,8,10,12]:
        days = 31
    elif month_mod in [4,6,9,11]:
        days = 30
    elif month_mod == 2:
        #if year % 4 == 0:
        #    days = 29
        #else:
            days = 28
    else:
        print("Not a valid month number")
        return None
    
    #p = 0
    #n = 0
    #balance = 0
    gain = 0

    pr_fac = yearly_increase**year * price
    
    #make sure if we don't have year or sqft they are set to proper values for default calc
    loss = month_demand_norm(month_mod,sqft,Year_Blt) * pr_fac
    
    for day in range(1,days+1):
        gain  = gain + days_yield(system,month_mod,day,daily_bool)
    
    gain = gain * pr_fac
    #balance = (gain-loss)*pr 

    #if balance >= 0:
    #    p = 1
    #else:
    #    n = 1

    this_month = State(P = gain, C = loss)
    return this_month
    
#############################################################################################

In [48]:
#############################################################################################

def update_fxn(state,system,month):
    '''Update the pos/neg/balance model.

    state: State with variables PB, FB, C
    system: System with relevant info
    '''
    p, c = state

    month_result = calc_month(system, month)

    p += month_result.P
    #n += month_result.N
    #pb += month_result.B
    #b += month_result.B
    c += month_result.C

    return State(P = p, C = c)

#############################################################################################

In [49]:
#############################################################################################

####   The function below generates three TimeSeries objects over the time interval specified 
###    within the provided time interval. The TimeSeries track the overall balance throughout 
#      the interval as well as the cost of electricity without a solar system
    
def run_simulation(system,upd_fxn):
    """Take a system as input and unpdate it based on the update function.

    system - system object defining panels
    update_fxn - function describing change to system 

    returns - Timeseries
    """
    P = TimeSeries()
    #N = TimeSeries()
    #PB = TimeSeries()
    #B = TimeSeries()
    C = TimeSeries()

    state = system.start
    t0 = system.t0
    P[t0], C[t0] = state

    for t in linrange(system.t0, system.t_end):
        state = upd_fxn(state,system,t)
        P[t+1], C[t+1] = state

    return P, -C
    
#############################################################################################

In [101]:
#%%timeit
#convert sq ft to sq m
roof_A = float(roof_sqft) * 0.092903
sim_years = 15
#############################################################################################
### equation for azimuth_factor below explained at:
##  https://docs.google.com/spreadsheets/d/13UL_QRR396G7L1IOa9wtQAjv24xCWA5So69pHfMEGI0/edit?usp=sharing
#   Equation of form: a + bx + cx^2 + dx^3 + ex^4
a = 0.995
b = 0
c = -2.69e-5
d = 0
e = 3.24e-10
azimuth_factor = a + (b*roof_azimuth) + (c*roof_azimuth**2) + (d*roof_azimuth**3) + (e*roof_azimuth**4)
#############################################################################################

low_r_bound = 0.175
high_r_bound = 0.2
low_initial_cost = 15000
high_initial_cost = 25000

cap_1 = "I=$" + str(int(high_initial_cost/1000)) + "k, r="+ str(low_r_bound)
cap_2 = "I=$" + str(int(high_initial_cost/1000)) + "k, r="+ str(high_r_bound)
cap_3 = "I=$" + str(int(low_initial_cost/1000)) + "k, r="+ str(low_r_bound)
cap_4 = "I=$" + str(int(low_initial_cost/1000)) + "k, r="+ str(high_r_bound)

#############################################################################################
### 4 systems below cover the range of initial costs (15k & 25k) as well as efficiency ranges (17.5% & 20%)
##  Can be adjusted to plot desired systems

system = define_system(A=roof_A, lat=la, long=lo, azim= azimuth_factor)
P, C = run_simulation(system,update_fxn)
FB = (P*low_r_bound - high_initial_cost) + C
FB2 = (P*high_r_bound - high_initial_cost) + C
FB3 = (P*low_r_bound - low_initial_cost) + C
FB4 = (P*high_r_bound - low_initial_cost) + C

### Combining the data from four systems above
projection = pd.concat([FB,FB2,FB3,FB4,C], axis =1)
#############################################################################################


#############################################################################################
### Column names below need to be either adjusted to match or automated to cover plotted systems

#!#!#!#! Need to deal with these names potentially changing along with the systems #!#!#!#

projection.columns = [cap_1,cap_2,cap_3,cap_4,'Regular Grid Service']
year = [i/12 for i in range(12*sim_years+1)]
projection['year'] = list(year)
projection.set_index('year', inplace = True)
#############################################################################################


#############################################################################################
#### Finding the intersection of each of the 4 systems with the 'Regular Grid Service'
###  'intersect' list contains 4 entries - the month at which each system intersects
##   Even_pt_hi/lo rounds the lowest and highest intersection points to years
#    Series names need to match column names above
intersect = []
find_60 = []
test1 = 1
test2 = 1
test3 = 1
test4 = 1
test60_1 = 1
test60_2 = 1
test60_3 = 1
test60_4 = 1
for i,r in projection.iterrows():
    if r[cap_1] > r['Regular Grid Service'] and test1:
        intersect.append(i)
        test1 = 0
    if r[cap_2] > r['Regular Grid Service'] and test2:
        intersect.append(i)
        test2 = 0
    if r[cap_3] > r['Regular Grid Service'] and test3:
        intersect.append(i)
        test3 = 0
    if r[cap_4] > r['Regular Grid Service'] and test4:
        intersect.append(i)
        test4 = 0
    if (r['Regular Grid Service']-r[cap_1])/25000 < 0.4 and test60_1:
        find_60.append(i)
        test60_1 = 0
    if (r['Regular Grid Service']-r[cap_2])/25000 < 0.4 and test60_2:
        find_60.append(i)
        test60_2 = 0
    if (r['Regular Grid Service']-r[cap_3])/15000 < 0.4 and test60_3:
        find_60.append(i)
        test60_3 = 0
    if (r['Regular Grid Service']-r[cap_4])/15000 < 0.4 and test60_4:
        find_60.append(i)
        test60_4 = 0
even_pt_lo = math.ceil(min(intersect)/12)
even_pt_hi = math.ceil(max(intersect)/12)
#############################################################################################


In [103]:
#############################################################################################
### Defining colormap for the lines - again, need to match column names from above
fig = px.line(projection,
    color_discrete_map={
        cap_1: 'blue',
        cap_2:'green',
        cap_3:'aqua',
        cap_4:'purple',
        'Regular Grid Service':'red'
    })
#############################################################################################
### Title and axis names for plot
fig.update_layout(
    title={
        'text':'Cost of regular grid power -vs- solar panel array',
        'x':0.5,
        'xanchor':'center',
        'yanchor':'top'
    },
    xaxis_title='# of Years',
    yaxis_title='Projected Cost ($)'
)
#############################################################################################
### Add an opaque section outlining the "Likely Break-even Range"
##  Use data from 'intersect' to determine x-range
fig.add_vrect(
    x0=min(intersect), x1=max(intersect),
    fillcolor='rgb(179,226,205)', opacity=0.5,
    layer="below", line_width=0
)

### Figure out location of the label for the range
max_earn = int(projection[cap_4].max())
ceiling = max(max_earn,0) - 2500
fig.update(layout=dict(
    annotations=[
        go.layout.Annotation(x=(min(intersect)+max(intersect))/2, 
            y=ceiling,
            text="Likely Break-Even Range",
            showarrow=False
        )
    ]
))
#############################################################################################
### Add lines at the 60% initial investment recoup
for x_60 in find_60:
    fig.add_vline(x=x_60, line_width=3, line_dash="dash", line_color="green")

### Draw figure
fig

In [72]:
"I=$" + str(int(low_initial_cost/1000)) + "k, r="+ str(low_r_bound)


'I=$15k, r=0.175'

In [104]:
projection

,"I=$25k, r=0.175","I=$25k, r=0.2","I=$15k, r=0.175","I=$15k, r=0.2",Regular Grid Service
year,,,,,
0.000000,-25000.000000,-25000.000000,-15000.000000,-15000.000000,-0.000000
0.083333,-25127.574896,-25116.455651,-15127.574896,-15116.455651,-205.409610
0.166667,-25180.702553,-25153.547253,-15180.702553,-15153.547253,-370.789650
0.250000,-25168.824161,-25117.648618,-15168.824161,-15117.648618,-527.052960
0.333333,-25142.010078,-25058.419232,-15142.010078,-15058.419232,-727.146000
...,...,...,...,...,...
14.666667,-13869.130762,-7274.500675,-3869.130762,2725.499325,-35031.541368
14.750000,-13665.845069,-7006.365125,-3665.845069,2993.634875,-35282.204675
14.833333,-13559.694617,-6859.338617,-3559.694617,3140.661383,-35462.186619
